# Integrantes

- Agustín Maglione  
- Fernando Sebastián Lomazzi  
- Juan Cruz Becerra  
- Maria Gabriela Bohórquez


## Versiones de entorno

In [ ]:
# Versión de Python
import sys
print(sys.version)

3.12.11 (main, Jun 26 2025, 21:20:05) [Clang 20.1.4 ]


In [ ]:
# Versiones de librerías principales
import numpy, scipy, sklearn, tqdm
print("numpy", numpy.__version__)
print("scipy", scipy.__version__)
print("scikit-learn", sklearn.__version__)
print("tqdm", tqdm.__version__)

numpy 2.3.1
scipy 1.16.1
scikit-learn 1.7.1
tqdm 4.67.1


# TP: LDA/QDA y optimización matemática de modelos


## Introducción teórica (resumen)

**Regla de Bayes.** Para clases con a priori $\pi_j$ y densidades condicionales $f_j$, se decide
$
H(x)=\arg\max_j \{\log f_j(x)+\log\pi_j\}.
$

**Modelo normal multivariado (QDA/LDA).**
$
\log f_j(x)=-\tfrac12\log|\Sigma_j|-\tfrac12(x-\mu_j)^\top\Sigma_j^{-1}(x-\mu_j)+C.
$
En **LDA**, $\Sigma_j=\Sigma$ para todo $j$ y
$
\log f_j(x)=\mu_j^\top\Sigma^{-1}\bigl(x-\tfrac12\mu_j\bigr)+C'.
$

**Estimación (MV).** $\hat\mu_j=\bar x_j$, $\hat\Sigma_j=s_j^2$, $\hat\pi_j=n_j/n$; en LDA, $\hat\Sigma=\frac1n\sum_j n_j s_j^2$.


In [ ]:

import numpy as np
import pandas as pd
import numpy.linalg as LA
from scipy.linalg import cholesky, solve_triangular
from scipy.linalg.lapack import dtrtri

from sklearn.datasets import load_iris, fetch_openml, load_wine
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


## Base de código (Profesor)

In [ ]:

class BaseBayesianClassifier:
    def __init__(self):
        pass

    def _estimate_a_priori(self, y):
        a_priori = np.bincount(y.flatten().astype(int)) / y.size
        # Q3: para qué sirve bincount?
        return np.log(a_priori)

    def _fit_params(self, X, y):
        # estimate all needed parameters for given model
        raise NotImplementedError()

    def _predict_log_conditional(self, x, class_idx):
        # log P(x|G=class_idx) según el modelo
        raise NotImplementedError()

    def fit(self, X, y, a_priori=None):
        # estimar probabilidades a priori si no se dan
        self.log_a_priori = self._estimate_a_priori(y) if a_priori is None else np.log(a_priori)
        # ajustar parámetros del modelo específico
        self._fit_params(X, y)
        # Q4: ¿por qué _fit_params va al final?

    def predict(self, X):
        # predicción individuo por individuo (loop externo)
        m_obs = X.shape[1]
        y_hat = np.empty(m_obs, dtype=int)
        for i in range(m_obs):
            y_hat[i] = self._predict_one(X[:, i].reshape(-1, 1))
        # devolver predicción como vector fila
        return y_hat.reshape(1, -1)

    def _predict_one(self, x):
        # calcular log-posteriori para todas las clases
        log_posteriori = [log_a_priori_i + self._predict_log_conditional(x, idx) 
                          for idx, log_a_priori_i in enumerate(self.log_a_priori)]
        # devolver la clase con posteriori máxima
        return np.argmax(log_posteriori)


In [ ]:
def get_iris_dataset():
    data = load_iris()
    X_full = data.data
    y_full = data.target.reshape(-1, 1)
    return X_full, y_full

def get_penguins_dataset():
    df, tgt = fetch_openml(name="penguins", return_X_y=True, as_frame=True, parser='auto')
    df.drop(columns=["island", "sex"], inplace=True)
    mask = df.isna().sum(axis=1) == 0
    df = df[mask]
    tgt = tgt[mask]
    return df.values, tgt.to_numpy().reshape(-1, 1)

def get_wine_dataset():
    data = load_wine()
    X_full = data.data
    y_full = data.target.reshape(-1, 1)
    return X_full, y_full

def get_letters_dataset():
    letter = fetch_openml('letter', version=1, as_frame=False)
    return letter.data, letter.target.reshape(-1, 1)

def label_encode(y_full):
    return LabelEncoder().fit_transform(y_full.flatten()).reshape(y_full.shape)

def split_transpose(X, y, test_size, random_state):
    # retornar X_train, X_test, y_train, y_test, todos transpuestos
    return [elem.T for elem in train_test_split(X, y, test_size=test_size, random_state=random_state)]


In [ ]:
import time
from tqdm.notebook import tqdm
from numpy.random import RandomState
import tracemalloc

RNG_SEED = 6553

class Benchmark:
    def __init__(self, X, y, n_runs=100, warmup=20, mem_runs=20, test_sz=0.3, rng_seed=RNG_SEED, same_splits=True):
        self.X = X
        self.y = y
        self.n = n_runs
        self.warmup = warmup
        self.mem_runs = mem_runs
        self.test_sz = test_sz
        self.det = same_splits
        if self.det:
            self.rng_seed = rng_seed
        else:
            self.rng = RandomState(rng_seed)

        self.data = dict()
        print("Benching params:")
        print("Total runs:", self.warmup + self.mem_runs + self.n)
        print("Warmup runs:", self.warmup)
        print("Peak Memory usage runs:", self.mem_runs)
        print("Running time runs:", self.n)
        approx_test_sz = int(self.y.size * self.test_sz)
        print("Train size rows (approx):", self.y.size - approx_test_sz)
        print("Test size rows (approx):", approx_test_sz)
        print("Test size fraction:", self.test_sz)

    def bench(self, model_class, **kwargs):
        name = model_class.__name__
        time_data = np.empty((self.n, 3), dtype=float)    # [train_time, test_time, accuracy]
        mem_data = np.empty((self.mem_runs, 2), dtype=float)  # [train_peak_mem, test_peak_mem]
        rng = RandomState(self.rng_seed) if self.det else self.rng

        # Warmup
        for i in range(self.warmup):
            model = model_class(**kwargs)
            X_train, X_test, y_train, y_test = split_transpose(self.X, self.y,
                                                               test_size=self.test_sz,
                                                               random_state=rng)
            model.fit(X_train, y_train)
            model.predict(X_test)

        # Mem runs
        for i in tqdm(range(self.mem_runs), total=self.mem_runs, desc=f"{name} (MEM)"):
            model = model_class(**kwargs)
            X_train, X_test, y_train, y_test = split_transpose(self.X, self.y,
                                                               test_size=self.test_sz,
                                                               random_state=rng)
            tracemalloc.start()
            t1 = time.perf_counter()
            model.fit(X_train, y_train)
            t2 = time.perf_counter()
            _, train_peak = tracemalloc.get_traced_memory()
            tracemalloc.reset_peak()
            model.predict(X_test)
            t3 = time.perf_counter()
            _, test_peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            mem_data[i, :] = (train_peak / (1024 * 1024), test_peak / (1024 * 1024))

        # Time runs
        for i in tqdm(range(self.n), total=self.n, desc=f"{name} (TIME)"):
            model = model_class(**kwargs)
            X_train, X_test, y_train, y_test = split_transpose(self.X, self.y,
                                                               test_size=self.test_sz,
                                                               random_state=rng)
            t1 = time.perf_counter()
            model.fit(X_train, y_train)
            t2 = time.perf_counter()
            preds = model.predict(X_test)
            t3 = time.perf_counter()
            time_data[i, :] = ((t2 - t1) * 1000,
                               (t3 - t2) * 1000,
                               (y_test.flatten() == preds.flatten()).mean())

        self.data[name] = (time_data, mem_data)

    def summary(self, baseline=None):
        aux = []
        for name, (time_data, mem_data) in self.data.items():
            result = {
                'model': name,
                'train_median_ms': np.median(time_data[:, 0]),
                'train_std_ms': time_data[:, 0].std(),
                'test_median_ms': np.median(time_data[:, 1]),
                'test_std_ms': time_data[:, 1].std(),
                'mean_accuracy': time_data[:, 2].mean(),
                'train_mem_median_mb': np.median(mem_data[:, 0]),
                'train_mem_std_mb': mem_data[:, 0].std(),
                'test_mem_median_mb': np.median(mem_data[:, 1]),
                'test_mem_std_mb': mem_data[:, 1].std()
            }
            aux.append(result)
        df = pd.DataFrame(aux).set_index('model')
        if baseline is not None and baseline in self.data:
            df['train_speedup'] = df.loc[baseline, 'train_median_ms'] / df['train_median_ms']
            df['test_speedup'] = df.loc[baseline, 'test_median_ms'] / df['test_median_ms']
            df['train_mem_reduction'] = df.loc[baseline, 'train_mem_median_mb'] / df['train_mem_median_mb']
            df['test_mem_reduction'] = df.loc[baseline, 'test_mem_median_mb'] / df['test_mem_median_mb']
        return df

In [ ]:
def comparar_implementaciones(
    get_datasets_fn=None,
    models=None,
    n_runs=10, warmup=3, mem_runs=3, test_sz=0.3, same_splits=False, baseline=None
):
    """
    Ejecuta Benchmark sobre un conjunto de modelos y devuelve/imprime el resumen.
    - get_dataset_fn: función que retorna (X_full, y_full)
    - models: lista de clases de modelo a evaluar.
    - *_runs: cantidad de corridas para warmup/mem/time; test_sz: fracción test;
    - same_splits: si True usa splits deterministas; baseline: nombre para speedup/reducción.
    """
    if get_datasets_fn is None:
        print("No se especificó get_dataset_fn")
        return
    if baseline is None:
        print("No se especificó baseline sobre el cual comparar los modelos")
        return
    if not isinstance(baseline, str):
        print("El parametro baseline debe ser un string con el nombre exacto del modelo")
        return
    results = {}
    for get_dataset in get_datasets_fn:
        X_full, y_full = get_dataset()
        y_enc = label_encode(y_full)
        b = Benchmark(X_full, y_enc, n_runs=n_runs, warmup=warmup, mem_runs=mem_runs,
                    test_sz=test_sz, same_splits=same_splits)
        if models is None:
            print("No se especificaron modelos")
            return
        for model in models:
            b.bench(model)
        df = b.summary(baseline=baseline)
        print(get_dataset.__name__, ": \n", df)
        results[get_dataset.__name__] = df
    return results

# Respuestas

# 1) Tensorización — Incisos 1 a 2

In [ ]:

class QDA(BaseBayesianClassifier):

    def _fit_params(self, X, y):
        # estimar matriz de covarianzas inversa para cada clase
        self.inv_covs = [LA.inv(np.cov(X[:, y.flatten() == idx], bias=True))
                         for idx in range(len(self.log_a_priori))]
        # medias por clase
        self.means = [X[:, y.flatten() == idx].mean(axis=1, keepdims=True)
                      for idx in range(len(self.log_a_priori))]

    def _predict_log_conditional(self, x, class_idx):
        inv_cov = self.inv_covs[class_idx]
        unbiased_x = x - self.means[class_idx]
        return 0.5 * np.log(LA.det(inv_cov)) - 0.5 * (unbiased_x.T @ inv_cov @ unbiased_x)


## Inciso 1 — ¿Sobre qué paraleliza `TensorizedQDA`?

**Respuesta:** Paraleliza sobre las $k$ clases (no sobre las $n$ observaciones). Para cada $x$ calcula $\log f_j(x)$ para todos los $j$ en una sola operación vectorizada; el loop sobre observaciones se mantiene.


In [ ]:

class TensorizedQDA(QDA):

    def _fit_params(self, X, y):
        super()._fit_params(X, y)
        # tensores apilados para vectorizar sobre clases
        self.tensor_inv_cov = np.stack(self.inv_covs)   # (k,p,p)
        self.tensor_means   = np.stack(self.means)      # (k,p,1)

    def _predict_log_conditionals(self, x):
        unbiased_x = x - self.tensor_means                          # (k,p,1)
        inner_prod = unbiased_x.transpose(0, 2, 1) @ self.tensor_inv_cov @ unbiased_x  # (k,1,1)
        return 0.5 * np.log(LA.det(self.tensor_inv_cov)) - 0.5 * inner_prod.flatten()

    def _predict_one(self, x):
        return np.argmax(self.log_a_priori + self._predict_log_conditionals(x))


## Inciso 2 — Shapes y equivalencia entre `QDA` y `TensorizedQDA`

`tensor_inv_cov: (k,p,p)` y `tensor_means: (k,p,1)`. 

Para un $x\in\mathbb{R}^{p\times 1}$, `unbiased_x = x - tensor_means` da $(k,p,1)$; 

el producto `unbiased_x^T @ tensor_inv_cov @ unbiased_x` produce $(k,1,1)$ con los valores $(x-\mu_j)^\top\Sigma_j^{-1}(x-\mu_j)$. 

Sumando $\log\pi_j$ se obtiene la misma predicción que `QDA`.


**Detalle de shapes (paso a paso)**  
- `x`: `(p, 1)`  
- `tensor_means`: `(k, p, 1)`  ⇒ `unbiased_x = x - tensor_means` ⇒ `(k, p, 1)`  
- `tensor_inv_cov`: `(k, p, p)`  
- `unbiased_x.transpose(0, 2, 1) @ tensor_inv_cov @ unbiased_x` ⇒ `(k, 1, 1)`  
- Aplanando, obtenemos `(k,)` con una log-verosimilitud por clase.



# 2) Optimización - Incisos 3 a 7

## Inciso 3 — Implementación `FasterQDA` (sin loops, pero ineficiente en memoria)

Vectoriza también sobre observaciones formando, para cada clase, la matriz $(X-\mu)^\top\Sigma^{-1}(X-\mu)$ de tamaño $n\times n$ y tomando su diagonal.


In [ ]:

class FasterQDA(TensorizedQDA):
    def _fit_params(self, X, y):
        super()._fit_params(X, y)
        # Precompute 0.5*log|Σ_k^{-1}| = -0.5*log|Σ_k|
        # Usamos self.tensor_inv_cov que es (K,p,p)
        self.half_logdet_inv = 0.5 * np.log(np.linalg.det(self.tensor_inv_cov))  # (K,)

    def predict(self, X):
        """
        Predice en batch sin bucles formando la matriz (n x n) por clase:
        X: (p, m) -> retorna (1, m)
        """
        # Dif de cada clase con cada columna de X: (K,p,m)
        diff = X[None, :, :] - self.tensor_means      # (K,p,1) broadcast -> (K,p,m)

        # Matmul batcheado sobre clases: (K,p,p) @ (K,p,m) -> (K,p,m)
        y = self.tensor_inv_cov @ diff

        # Matriz completa Q_full = (X-μ)^T Σ^{-1} (X-μ) de tamaño (m x m) por clase
        Q_full = diff.transpose(0, 2, 1) @ y   # (K,m,m)

        # Diagonal (una cuadrática por observación)
        quad = np.diagonal(Q_full, axis1=1, axis2=2)     # (K,m)

        # Puntajes g_k(x) = log π_k + 0.5*log|Σ_k^{-1}| - 0.5*quad
        scores = self.log_a_priori[:, None] + self.half_logdet_inv[:, None] - 0.5 * quad  # (K,m)

        # Clase argmax por columna
        return scores.argmax(axis=0, keepdims=True)   # (1,m)


## Inciso 4 — ¿Dónde aparece la matriz $n\times n$? ¿Cómo evitarla?

Para una clase fija $k$, sea:

- $D_k = X - μ_k$ con $X$ $(p\times n)$ y $μ_k$ $(p\times 1)$ (broadcast) $→ D_k$ es $(p\times n)$.

- Cuadrática por observación: queremos el vector $d(x_i) = (x_i−μ_k)^T Σ_k^{-1} (x_i−μ_k)$ de tamaño $n$.

Si hacemos la vectorización con `@`:

$Q_{kfull} = D_k.T$ @ $Σ_k^{-1}$ @ $D_k →$ matriz $(n\times n)$.

Cada entrada $(i,j) = (x_i−μ_k)^T Σ_k^{-1} (x_j−μ_k)$.

Lo que necesitamos son solo las diagonales: $\operatorname{diag}(Q_{kfull}) = d(x_i)$.

Esto muestra explícitamente la matriz $(n\times n)$.

---

Cómo lo evitamos en `FasterQDA`:

Implementamos:
- $Y_k = Σ_k^{-1} @ D_k$ $(p×n)$,
- $d = (D_k * Y_k).sum(axis=1)$ (vector $n$),

Sin nunca formar $Q_{kfull}$ $(n×n)$.

En el código anterior, esto es exactamente:

`y = tensor_inv_cov @ diff` y luego `quad = (diff * y).sum(axis=1)`.

## Inciso 5 - Demostración 

**Prueba por componentes (índice a índice).** 

La entrada $(i,i)$ de $AB$ es $(AB)_{ii}=\sum_{k=1}^p A_{ik}B_{ki}$. 

Como $(B^\top)_{ik}=B_{ki}$, se tiene $(A⊙B^\top)_{ik}=A_{ik}(B^\top)_{ik}=A_{ik}B_{ki}$. 

Al sumar por columnas (en $k$) la fila $i$ de $A⊙B^\top$, obtenemos $\sum_{k=1}^p (A⊙B^\top)_{ik}=(AB)_{ii}$. 

Por lo tanto, $diag⁡(AB)=\sum_{\text{cols}}(A\odot B^\top)=np.sum(A \odot B.T, axis=1)$.

---

**Por qué “esquivamos” la matriz $n\times n$.**

Formar $AB$ completo para luego tomar solo la diagonal requiere memoria $O(n^2)$ y tiempo $O(n^2p)$. 

En cambio, calcular directamente $np.sum(A \odot B.T, axis=1)$ nunca construye la matriz $n\times n$, usa solo objetos $n\times p$ (más un vector) y añade $O(np)$ operaciones, habilitando la versión eficiente.

## Inciso 6 - Propiedad clave (para `EfficientQDA`) — sin formar la matriz $n\times n$

Para **evitar la matriz $n\times n$**, usamos la identidad
$\operatorname{diag}(AB)=\sum (A\odot B^T)$ y obtenemos directamente la diagonal con operaciones elemento a elemento. 

Implementado en `EfficientQDA`:

In [ ]:

class EfficientQDA(TensorizedQDA):
    def predict(self, X):
        # Vectorizar sobre clases y observaciones sin formar (n x n)
        unbiased_X = X[np.newaxis, :, :] - self.tensor_means   # (k,p,n)
        B = self.tensor_inv_cov @ unbiased_X         # (k,p,n)
        cond_quad = np.sum(unbiased_X * B, axis=1)             # (k,n)  diag((X-mu)^T Sig^-1 (X-mu))
        log_det_terms = 0.5 * np.log(np.linalg.det(self.tensor_inv_cov))  # (k,)
        log_cond_matrix = log_det_terms[:, None] - 0.5 * cond_quad        # (k,n)
        log_post = log_cond_matrix + self.log_a_priori[:, None]            # (k,n)
        return np.argmax(log_post, axis=0).reshape(1, -1)


## Inciso 7 - Comparacion QDA (en sus 4 versiones)

In [ ]:
comparacion_4_variantes = comparar_implementaciones(
        get_dataset_fn=[get_iris_dataset, get_wine_dataset, get_penguins_dataset, get_letters_dataset],
        models=[QDA, TensorizedQDA, FasterQDA, EfficientQDA],
        n_runs=5, warmup=2, mem_runs=2, test_sz=0.3, same_splits=False, baseline='QDA'
)

# 3) Cholesky (Comparaciones) — Incisos 8 a 11

In [ ]:

class TensorizedChol(BaseBayesianClassifier):
    """QDA con Cholesky vectorizado sobre observaciones.
    - Ajuste: guarda L_k y mu_k por clase, y suma de log(diag(L_k)).
    - Predicción: resuelve L_k Y = (X - mu_k) para TODAS las columnas de X.
    """
    def _fit_params(self, X, y):
        self.Ls = [
            cholesky(np.cov(X[:, y.flatten() == idx], bias=True), lower=True)
            for idx in range(len(self.log_a_priori))
        ]
        self.means = [X[:, y.flatten() == idx].mean(axis=1, keepdims=True)
                      for idx in range(len(self.log_a_priori))]
        self.sumlog_diagL = np.array([np.log(L.diagonal()).sum() for L in self.Ls])

    def predict(self, X):
        k, n = len(self.log_a_priori), X.shape[1]
        log_cond = np.empty((k, n))
        for idx, L in enumerate(self.Ls):
            unbiased_X = X - self.means[idx]            # (p,n)
            Y = solve_triangular(L, unbiased_X, lower=True)  # (p,n)
            quad = (Y ** 2).sum(axis=0)                 # (n,)
            log_cond[idx, :] = -self.sumlog_diagL[idx] - 0.5 * quad
        log_post = log_cond + self.log_a_priori[:, None]
        return np.argmax(log_post, axis=0).reshape(1, -1)


class EfficientChol(BaseBayesianClassifier):
    """QDA con Cholesky + matriz triangular invertida precomputada (vectorizado).
    - Ajuste: L_k^{-1} por clase vía LAPACK (dtrtri), y mu_k.
    - Predicción: Y = L_k^{-1} (X - mu_k) para TODAS las columnas (producto matricial).
    """
    def _fit_params(self, X, y):
        self.L_invs = [
            dtrtri(cholesky(np.cov(X[:, y.flatten() == idx], bias=True), lower=True), lower=1)[0]
            for idx in range(len(self.log_a_priori))
        ]
        self.means = [X[:, y.flatten() == idx].mean(axis=1, keepdims=True)
                      for idx in range(len(self.log_a_priori))]
        # log |Σ|^{-1/2} = - sum log diag(L) = sum log diag(L^{-1})
        self.sumlog_diagLinv = np.array([np.log(Li.diagonal()).sum() for Li in self.L_invs])

    def predict(self, X):
        k, n = len(self.log_a_priori), X.shape[1]
        log_cond = np.empty((k, n))
        for idx, L_inv in enumerate(self.L_invs):
            unbiased_X = X - self.means[idx]            # (p,n)
            Y = L_inv @ unbiased_X                      # (p,n)
            quad = (Y ** 2).sum(axis=0)                 # (n,)
            log_cond[idx, :] = self.sumlog_diagLinv[idx] - 0.5 * quad
        log_post = log_cond + self.log_a_priori[:, None]
        return np.argmax(log_post, axis=0).reshape(1, -1)

In [ ]:

class QDA_Chol1(BaseBayesianClassifier):

    def _fit_params(self, X, y):
        self.L_invs = [
            LA.inv(cholesky(np.cov(X[:, y.flatten() == idx], bias=True), lower=True))
            for idx in range(len(self.log_a_priori))
        ]
        self.means = [X[:, y.flatten() == idx].mean(axis=1, keepdims=True)
                      for idx in range(len(self.log_a_priori))]

    def _predict_log_conditional(self, x, class_idx):
        L_inv = self.L_invs[class_idx]
        unbiased_x = x - self.means[class_idx]
        y = L_inv @ unbiased_x
        return np.log(L_inv.diagonal().prod()) - 0.5 * (y ** 2).sum()


class QDA_Chol2(BaseBayesianClassifier):

    def _fit_params(self, X, y):
        self.Ls = [
            cholesky(np.cov(X[:, y.flatten() == idx], bias=True), lower=True)
            for idx in range(len(self.log_a_priori))
        ]
        self.means = [X[:, y.flatten() == idx].mean(axis=1, keepdims=True)
                      for idx in range(len(self.log_a_priori))]

    def _predict_log_conditional(self, x, class_idx):
        L = self.Ls[class_idx]
        unbiased_x = x - self.means[class_idx]
        y = solve_triangular(L, unbiased_x, lower=True)
        return -np.log(L.diagonal().prod()) - 0.5 * (y ** 2).sum()


class QDA_Chol3(BaseBayesianClassifier):

    def _fit_params(self, X, y):
        self.L_invs = [
            dtrtri(cholesky(np.cov(X[:, y.flatten() == idx], bias=True), lower=True), lower=1)[0]
            for idx in range(len(self.log_a_priori))
        ]
        self.means = [X[:, y.flatten() == idx].mean(axis=1, keepdims=True)
                      for idx in range(len(self.log_a_priori))]

    def _predict_log_conditional(self, x, class_idx):
        L_inv = self.L_invs[class_idx]
        unbiased_x = x - self.means[class_idx]
        y = L_inv @ unbiased_x
        return np.log(L_inv.diagonal().prod()) - 0.5 * (y ** 2).sum()

## Inciso 8 — Si $A=LL^\top$, expresar $A^{-1}$ y utilidad en QDA

$A^{-1}=(L^\top)^{-1}L^{-1}$. 

Entonces $(x-\mu)^\top\Sigma^{-1}(x-\mu)=\|L^{-1}(x-\mu)\|^2$ y $\tfrac12\log|\Sigma^{-1}|=\log|L^{-1}|$. 

La utilidad en QDA radica en que permite evitar invertir $\Sigma$ y resolver sistemas triangulares más estables y rápidos.

## Inciso 9 — Diferencias `QDA` vs `QDA_Chol1` y paso a paso de predicción
#### Entrenamiento:

- `QDA`:
    - Estima $Σ_j$ con `np.cov(…, bias=True)` y guarda su inversa completa: `inv_covs[j] = LA.inv(Σ_j)`.
    - Guarda medias: `means[j]`.
- `QDA_Chol1`:
    - Factoriza $Σ_j = L_j L_j^T$ con `cholesky(Σ_j, lower=True)` y guarda la inversa triangular: `L_invs[j] = LA.inv(L_j)`.
    - Guarda medias: `means[j]`.

#### Predicción (paso a paso) en `QDA_Chol1._predict_log_conditional(x, j)`:
1. Centrado: `unbiased_x = x − means[j]`.
2. Blanqueo triangular: `y = L_invs[j] @ unbiased_x` (equivalente a resolver $L_j$ `y = unbiased_x`).
3. Forma cuadrática: `-0.5 ||y||^2 = -0.5 * (y**2).sum()`, que es $-0.5 (x − μ)^T Σ_j^{-1} (x − μ)$.
4. Log-det: `np.log(L_inv.diagonal().prod())`. Para triangular, $diag(L_inv) = 1/diag(L)$, por lo que este término vale $−∑ log diag(L) = 0.5 log|Σ_j^{-1}|$.
5. Log-condicional: suma de (4) y (3).
6. Posterior y clase: se suma `log_a_priori[j]` y se toma `argmax` (heredado de `BaseBayesianClassifier._predict_one`).

#### Resumen de diferencia clave:
- `QDA` usa la inversa completa $Σ_j^{-1}$ y el determinante de esa inversa.
- `QDA_Chol1` evita invertir $Σ_j$ completa: trabaja con la inversa triangular $L_j^{-1}$ y productos/diagonales de matrices triangulares (mejor condicionado que invertir $Σ$).


## Inciso 10 — Diferencias entre QDA_Chol1, QDA_Chol2 y QDA_Chol3

#### Qué guarda cada uno (fit):
- `QDA_Chol1`: guarda $L_j^{-1}$ (“inversa triangular”) tras Cholesky. Lo hace usando `L_invs = [LA.inv(cholesky(...))]`.
- `QDA_Chol2`: guarda $L_j$ (no su inversa). Lo hace usando `Ls = [cholesky(...)]`.
- `QDA_Chol3`: guarda $L_j^{-1}$ usando LAPACK especializado. Lo hace usando `L_invs = [dtrtri(cholesky(...), lower=1)[0]]`.

#### Cómo predicen:
- `QDA_Chol1`: $y = L_j^{-1}(x − μ_j)$; $log-det = log(∏ diag(L_j^{-1}))$; $log-cond = log-det − 0.5||y||^2$.
- `QDA_Chol2`: resuelve $L_j y = (x − μ_j)$ con solve_triangular (sin invertir); $log-det = −log(∏ diag(L_j))$; $log-cond = log-det − 0.5||y||^2$.
- `QDA_Chol3`: igual a Chol1, pero obtiene $L_j^{-1}$ de forma más eficiente/estable con dtrtri (mejor que `LA.inv` general para triangulares).

#### Estabilidad numérica:
- Mejor: `QDA_Chol2` (no hay inversión; solo solve triangular).
- Intermedia: `QDA_Chol3` (inversión triangular con LAPACK; más estable/rápida que invertir una matriz densa).
- Peor de los tres: `QDA_Chol1` (inversión triangular con `LA.inv`, aun así bastante mejor que invertir la matriz $Σ$ completa).

#### Costos (orden aproximado):
- Cholesky por clase: $O(p^3)$ (común a los tres).
- Invertir triangular (`Chol1`/`Chol3`): $O(p^3)$ adicional al fit; predicción luego es $O(p^2)$ por observación (multiplicar $L^{-1} @ v$).
- Resolver triangular (`Chol2`): $O(p^2)$ por observación sin costo de inversión; suele ser la opción más estable y competitiva en tiempo.

#### Log-det:
- `Chol1`/`Chol3`: usan $log(∏ diag(L^{-1})) = −∑ log diag(L) = 0.5 log|Σ^{-1}|$.
- `Chol2`: computa directamente $−log(∏ diag(L)) = −∑ log diag(L)$, que es el mismo valor.

#### Cuándo preferir:
- En casos de muchas predicciones por cada ajuste y $p$ no muy grande: precomputar $L^{-1}$ (`Chol3` > `Chol1`) puede acelerar por evitar múltiples “solves”.
- En casos de estabilidad prioritaria y/o $p$ moderado/grande: `Chol2`.

## Inciso 11 — Comparación breve de performance esperada

Esta sección agrega una función utilitaria que corre el Benchmark sobre las variantes de QDA
(`QDA`, `TensorizedQDA`, `QDA_Chol1`, `QDA_Chol2`, `QDA_Chol3`) y muestra un resumen.

En *fit*, `Chol2` suele ser más rápido (no invierte). 

En *predict*, las tres son similares ($O(p^2)$ por observación). 

En datasets con $p$ grande o covarianzas mal condicionadas, `Chol2` es preferible por estabilidad.


In [ ]:
# Ejecutar comparación rápida (Iris) y mostrar resumen al final del notebook
comparacion_7_variantes = comparar_implementaciones(
        get_dataset_fn=[get_iris_dataset, get_penguins_dataset, get_wine_dataset, get_letters_dataset],
        models=[QDA, TensorizedQDA, FasterQDA, EfficientQDA, QDA_Chol1, QDA_Chol2, QDA_Chol3],
        n_runs=5, warmup=2, mem_runs=2, test_sz=0.3, same_splits=False, baseline='QDA'
)

# 4) Cholesky (Optimización) — Incisos 12 a 14

12. Implementar el modelo `TensorizedChol` paralelizando sobre clases/observaciones según corresponda. Se recomienda heredarlo de alguna de las implementaciones de `QDA_Chol`, aunque la elección de cuál de ellas queda a cargo del alumno según lo observado en los benchmarks de puntos anteriores.
13. Implementar el modelo `EfficientChol` combinando los insights de `EfficientQDA` y `TensorizedChol`. Si se desea, se puede implementar `FasterChol` como ayuda, pero no se contempla para el punto.
14. Comparar la performance de las 9 variantes de QDA implementadas ¿Qué se observa? A modo de opinión ¿Se condice con lo esperado?

## Inciso 12 - Modelo `TensorizedChol`

In [ ]:
class TensorizedChol(QDA_Chol2):
    """QDA con Cholesky heredando de QDA_Chol2.
    - Ajuste: usa L_k y mu_k de QDA_Chol2 y precomputa sum(log diag L_k).
    - Predicción: vectorizado sobre observaciones (resuelve L_k Y = X - mu_k por clase).
    """
    def _fit_params(self, X, y):
        super()._fit_params(X, y)
        # 0.5*log|Sigma^{-1}| = -sum log diag(L)  (porque |Sigma| = (prod diag L)^2)
        self.sumlog_diagL = np.array([np.log(L.diagonal()).sum() for L in self.Ls])

    def predict(self, X):
        k, n = len(self.log_a_priori), X.shape[1]
        log_cond = np.empty((k, n))
        for idx, L in enumerate(self.Ls):
            unbiased_X = X - self.means[idx]                      # (p,n)
            # resolver L Y = unbiased_X sin formar inversas
            Y = solve_triangular(L, unbiased_X, lower=True, check_finite=False)
            quad = (Y ** 2).sum(axis=0)                           # (n,)
            log_cond[idx, :] = -self.sumlog_diagL[idx] - 0.5 * quad
        log_post = log_cond + self.log_a_priori[:, None]
        return np.argmax(log_post, axis=0).reshape(1, -1)

## Inciso 13 - Modelo `EfficientChol`

In [ ]:
class EfficientChol(TensorizedChol):
    """QDA con Cholesky eficiente heredando de TensorizedChol.
    - Combina ideas de EfficientQDA (sin n×n) + tensorización sobre clases.
    - Ajuste: computa L_k^{-1} (vía dtrtri) y apila tensores para vectorizar (k,p,p).
    - Predicción: Y = L_k^{-1} (X - mu_k) vectorizado sobre clases y observaciones.
    """
    def _fit_params(self, X, y):
        super()._fit_params(X, y)  # provee self.Ls, self.means, self.sumlog_diagL
        # Inversa triangular inferior por clase (más estable/eficiente que invertir Sigma)
        self.L_invs = [dtrtri(L, lower=1, overwrite_c=0)[0] for L in self.Ls]
        # Tensores apilados para vectorizar sobre clases
        self.tensor_L_inv = np.stack(self.L_invs)   # (k,p,p)
        self.tensor_means = np.stack(self.means)    # (k,p,1)
        # 0.5*log|Sigma^{-1}| = sum log diag(L^{-1})
        self.sumlog_diagLinv = np.array([np.log(Li.diagonal()).sum() for Li in self.L_invs])

    def predict(self, X):
        # Vectorizamos sobre clases (k) y observaciones (n) sin formar (n×n)
        unbiased_X = X[np.newaxis, :, :] - self.tensor_means     # (k,p,n)
        Y = self.tensor_L_inv @ unbiased_X                        # (k,p,n)
        cond_quad = (Y ** 2).sum(axis=1)                          # (k,n)
        log_post = (self.log_a_priori[:, None]
                    + self.sumlog_diagLinv[:, None]
                    - 0.5 * cond_quad)                           # (k,n)
        return np.argmax(log_post, axis=0).reshape(1, -1)


## Inciso 14 - Performance de las 9 variantes de QDA

In [ ]:
# Ejecutar comparación rápida (Iris) y mostrar resumen al final del notebook
comparacion_9_variantes = comparar_implementaciones(
        get_dataset_fn=[get_iris_dataset, get_penguins_dataset, get_wine_dataset, get_letters_dataset],
        models=[QDA, TensorizedQDA, FasterQDA, EfficientQDA, QDA_Chol1, QDA_Chol2, QDA_Chol3, TensorizedChol, EfficientChol],
        n_runs=5, warmup=2, mem_runs=2, test_sz=0.3, same_splits=False, baseline='QDA'
)